
# From Ground Motion to Digital Counts: Instrument + Digitizer as a Filter Chain  
## Example A: Trillium Compact Posthole 120s + Nanometrics Centaur  
## Example B: Raspberry Shake (typical RS4D / RS3D-class architecture)

This notebook builds an **LTI (linear time-invariant)** view of a seismic recording system as a **cascade of filters**.

You will learn to:
1. Write the end-to-end **convolution model** from ground motion to recorded counts for:
   - **(A)** *Trillium Compact Posthole 120s + Centaur* (observatory-grade)
   - **(B)** *Raspberry Shake* (low-cost / educational / citizen-science grade)
2. Interpret the **sensor** (Trillium) as a **transfer function** (poles/zeros + gain)
3. Interpret the **digitizer** (Centaur) as a set of **analog + digital stages**
4. Describe the Raspberry Shake as a set of stages (sensor + analog front-end + ADC + decimation, etc.)
5. Compare the two systems: noise, dynamic range, bandwidth, timing, metadata/response quality

> Pedagogical note  
> We’ll use **analog intuition** (transfer functions, convolution), but measurements are sampled and processed digitally.



## 0) Setup (optional code utilities)

This notebook is mostly conceptual, but includes optional code to:
- Load a StationXML Inventory
- Plot instrument response magnitude/phase
- Plot poles & zeros when metadata include PAZ stages

If you have ObsPy:
```bash
conda install -c conda-forge obspy matplotlib numpy
```


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Optional: response inspection with ObsPy
try:
    from obspy import read_inventory, UTCDateTime
    HAVE_OBSPY = True
except Exception:
    HAVE_OBSPY = False



## 1) System overview and the convolution model

### 1.1 LTI cascade model (continuous time)

Let ground motion be (choose one):
- displacement: \(u(t)\) [m]
- velocity: \(\dot{u}(t)\) [m/s]
- acceleration: \(\ddot{u}(t)\) [m/s\(^2\)]

A general end-to-end LTI model for *counts* \(y(t)\) is:

$$
y(t) = \bigl(h_{\text{total}} * x\bigr)(t)
$$

where \(x(t)\) is the chosen ground-motion input (e.g., velocity), and

$$
h_{\text{total}}(t) = h_{\text{sensor}}(t) * h_{\text{digitizer}}(t)
$$

In the frequency domain:

$$
H_{\text{total}}(f) = H_{\text{sensor}}(f)\,H_{\text{digitizer}}(f)
$$

### 1.2 Digitizer as a product of stages

A digitizer is rarely “transparent.” A common decomposition is:

$$
H_{\text{digitizer}}(f)
= H_{\text{AFE}}(f)\,
  H_{\text{AA}}(f)\,
  H_{\text{S\&H}}(f)\,
  H_{\text{ADC}}(f)\,
  H_{\text{dec}}(f)
$$

- **AFE**: analog front-end (buffering, programmable gain/attenuation, protection)
- **AA**: anti-alias analog lowpass
- **S&H**: sample-and-hold behavior (often rolled into ADC model)
- **ADC**: quantization + (often) oversampling sigma-delta dynamics
- **dec**: digital decimation FIR/IIR stages to produce final sample rate(s)

### 1.3 Discrete-time view (what we actually store)

In sampled form:

- sample interval: \(\Delta t = 1/f_s\)
- samples: \(x[n] = x(n\Delta t)\)
- recorded counts: \(y[n]\)

The cascade becomes:

$$
y[n] = (h[n] * x[n]) = \sum_{k=-\infty}^{\infty} h[k]\,x[n-k]
$$

When we do **response removal**, we attempt to invert \(H_{\text{total}}\) over the usable band.



## 2) Example A: Trillium Compact Posthole 120s + Centaur (observatory-grade)

### 2.1 Block diagram (conceptual)

```
ground motion  →  Trillium mechanical+electronics  →  differential voltage  →  Centaur AFE
                 (sensor transfer function)                                 ↓
                                                                          anti-alias
                                                                             ↓
                                                                          ADC (σ-Δ)
                                                                             ↓
                                                                       FIR decimation
                                                                             ↓
                                                                          counts
```

Key idea:
- The **Trillium sensor includes internal electronics (preamp)** that are part of the sensor response.
- The **Centaur** is primarily a precision digitizer with known anti-alias + digital decimation behavior.

### 2.2 End-to-end convolution equation for this system

Choose a convention: broadband seismometers are commonly specified as **velocity sensors** over their passband.

Let \(v(t)=\dot{u}(t)\) be ground velocity, then:

$$
y(t) = \bigl(h_{\text{Centaur}} * h_{\text{Trillium}} * v\bigr)(t)
$$

or in frequency domain:

$$
Y(f) = H_{\text{Centaur}}(f)\,H_{\text{Trillium}}(f)\,V(f)
$$

### 2.3 Practical metadata view (StationXML stages)

In StationXML/RESP terms you often see:

- Stage 1: **Sensor PAZ** (Trillium poles/zeros + sensitivity)
- Stage 2: Analog gain (Centaur input gain)
- Stage 3+: Digital FIR decimation stages (Centaur)
- Sometimes: explicit analog anti-alias stage, sometimes folded into stage gains

If metadata are complete, the product of all stages equals \(H_{\text{total}}\).



## 3) The Trillium Compact Posthole 120s as a set of filters (more detail)

A broadband seismometer can be modeled as:

1. **Mechanical oscillator** (mass-spring-damper)
2. **Electromechanical transduction** (velocity-proportional coil output, or capacitive sensing, etc.)
3. **Feedback / damping electronics** (force-balance / control loop, depending on design)
4. **Output electronics (preamp, buffering, differential driver)**

### 3.1 Canonical “sensor transfer function” picture

Most broadband sensors are supplied with a poles/zeros model:

$$
H_{\text{sensor}}(s) = K \frac{\prod_i (s - z_i)}{\prod_j (s - p_j)}
$$

or in frequency \(s = i2\pi f\):

$$
H_{\text{sensor}}(f) = K \frac{\prod_i (i2\pi f - z_i)}{\prod_j (i2\pi f - p_j)}
$$

**Interpretation**
- Poles close to the origin set long-period behavior (e.g., “120 s corner”).
- High-frequency poles control roll-off and stability.
- Zeros often appear at the origin for velocity/displacement conventions.

### 3.2 What the sensor electronics (preamp) change

The internal electronics are not “just gain”:
- set output impedance and drive long cables
- set noise floor (input-referred noise)
- may contribute poles/zeros (e.g., roll-off, stability shaping)
- define clipping behavior and recovery

So for teaching, it’s reasonable to think:

$$
H_{\text{Trillium}}(f) \approx H_{\text{mech}}(f)\,H_{\text{elec}}(f)
$$

and \(H_{\text{elec}}\) includes the internal preamp and feedback/damping dynamics.

### 3.3 Minimum-phase vs linear/zero-phase in sensors?

The sensor response is a **causal physical system** → it is effectively **minimum-phase-like** (no acausal behavior).
Any “zero-phase” you see is from **offline digital filtering**, not the sensor.



## 4) The Nanometrics Centaur as a set of filter stages (more detail)

Digitizers like the Centaur typically use:
- high-stability clocks (timing accuracy)
- low-noise differential inputs
- sigma-delta ADCs with oversampling
- multi-stage **digital decimation FIR** filters

A useful teaching decomposition:

### 4.1 Analog front-end (AFE)
- input protection
- differential buffering
- programmable gain/attenuation (to match sensor full-scale to ADC full-scale)
- very high input impedance (avoid loading the sensor)

Often modeled as approximately constant gain in-band:
$$
H_{\text{AFE}}(f) \approx G
$$

### 4.2 Analog anti-alias filter
- prevents energy above Nyquist (of the ADC modulator / first stage) from folding into band
- contributes amplitude + phase shaping near the corner

Modeled as a lowpass:
$$
H_{\text{AA}}(f) \approx \text{LP}(f; f_c, \text{order})
$$

### 4.3 Sigma-delta ADC + decimation
Sigma-delta converters:
- sample at very high internal rates (oversampling)
- shape quantization noise out of band
- rely on **digital FIR** filters to decimate to the requested output sample rate(s)

So the effective digital stage is:
$$
H_{\text{dec}}(f) = \prod_{m=1}^{M} H_{\text{FIR},m}(f)
$$

These FIR stages can have:
- very flat passbands
- very steep stopbands
- known group delay (latency), often constant within band (linear-phase FIR)

### 4.4 Latency / timing
- The system is causal; real-time output has latency.
- FIR decimation filters often have near-constant group delay; time stamping is usually corrected so sample times represent the correct physical time.
- For **phase picking**, you care about *relative* timing and phase distortion introduced by any additional *user-applied* filters.



## 5) Example B: Raspberry Shake as a set of filter stages (more detail)

Raspberry Shake systems vary by model. Conceptually, many share:

### 5.1 Sensor element
Common possibilities:
- a **geophone element** (velocity transducer) (e.g., vertical-only in some models)
- a **MEMS accelerometer** (acceleration-proportional)
- sometimes multiple channels (3D) depending on model

So the sensor stage might be:
- geophone: band-limited velocity response (natural frequency typically several Hz)
- MEMS: flat-ish acceleration response in some band, with its own noise characteristics

### 5.2 Analog front-end
- inexpensive but functional preamp/buffer
- anti-alias filtering (often simpler/less sharp than observatory-grade)
- input range and clipping may be more limiting

### 5.3 ADC
- typically lower-cost ADC than observatory digitizers
- dynamic range and noise floor often limiting compared to broadband + high-end digitizer

### 5.4 Digital processing / decimation
- digital filtering and decimation to output sample rate(s)
- timing derived from the host system (e.g., GNSS in some configurations, NTP in others)

### 5.5 End-to-end convolution equation (Raspberry Shake)

Because the sensor differs by model, write generically:

$$
y(t) = \bigl(h_{\text{RS, digi}} * h_{\text{RS, sensor}} * x\bigr)(t)
$$

with \(x(t)\) being velocity or acceleration depending on the sensor.

In frequency form:

$$
Y(f) = H_{\text{RS, digi}}(f)\,H_{\text{RS, sensor}}(f)\,X(f)
$$

### 5.6 What often matters most for Raspberry Shake data
- higher self-noise (especially at long periods)
- limited bandwidth (geophone natural frequency limits low-frequency fidelity)
- more sensitivity to installation/tilt/temperature
- response metadata can be available, but may be less detailed/less standardized than observatory-grade deployments



## 6) Key differences: Observatory-grade (Trillium+Centaur) vs Raspberry Shake

### 6.1 Bandwidth / usable period range
- **Trillium 120s**: designed for long periods (down to ~120 s) and broadband response
- **Raspberry Shake geophone-based**: usually cannot capture true long-period motion (limited below a few Hz)

### 6.2 Noise floor / self-noise
- Observatory-grade systems aim to be **sensor-noise-limited** across a wide band
- Raspberry Shake systems are often **electronics-noise-limited** and/or sensor-limited, especially at long periods

### 6.3 Dynamic range and clipping
- Centaur-class digitizers offer very large dynamic range and robust clipping behavior
- Raspberry Shake has more limited full-scale input and may clip more readily for strong nearby signals

### 6.4 Timing accuracy and stability
- Observatory-grade: disciplined clocks, precise timing, stable sampling
- Raspberry Shake: timing can be excellent if GNSS-disciplined, but may be more variable depending on configuration

### 6.5 Metadata quality / response removal
- Observatory-grade deployments typically have complete responses (PAZ + FIR stages) enabling accurate response removal
- Raspberry Shake responses may be available but may have:
  - fewer documented stages
  - simplified response models
  - model-dependent variations (sensor/ADC revisions)

### 6.6 Use cases (what each is good for)
**Trillium+Centaur**
- regional/teleseismic waveform analysis
- moment tensors, spectral methods, long-period studies
- high-quality phase picking and amplitude accuracy

**Raspberry Shake**
- event detection, education, outreach, citizen science
- local-to-regional monitoring where long-period fidelity is less critical
- dense low-cost arrays (trade quality for quantity)

### 6.7 A “rules of thumb” summary
- If you need **accurate long-period ground motion**, you need observatory-grade broadband instrumentation.
- If you need **lots of stations** cheaply for detection/education, Raspberry Shake is powerful.



## 7) Optional: Inspect response from StationXML (ObsPy)

If you have StationXML for your instruments, you can explore:
- overall response magnitude/phase
- stage breakdown
- poles and zeros (if PAZ stages are included)

Below are template cells you can adapt.


In [ ]:

# --- OPTIONAL: load a StationXML and plot response ---
# Provide your StationXML path(s) here.
# Example:
# inv = read_inventory("YOUR_STATIONXML.xml")

if not HAVE_OBSPY:
    print("ObsPy not available in this environment. Install obspy to use these response utilities.")


In [ ]:

# Example response plot (requires inv and matching codes)
# inv.plot_response(min_freq=0.001, output="VEL")  # or "DISP" / "ACC"
pass


In [ ]:

# Example: list stages and print basic info (requires inv)
# for net in inv:
#     for sta in net:
#         for cha in sta:
#             print(net.code, sta.code, cha.code, "stages:", len(cha.response.response_stages))
pass



## 8) Optional: Link back to datasheets

For the Centaur, the official datasheet describes:
- input configuration and full-scale ranges
- sample rates and decimation behavior
- timing and precision features
- (sometimes) response/latency details

For teaching: treat the datasheet as the authoritative source for the digitizer stage model,
and the StationXML/RESP as the authoritative source for *response removal*.
